In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.models import resnet18
import numpy as np
import random
import os
import urllib.request
import zipfile
import ssl


# Global config
LOCAL_OR_COLAB = "COLAB"

SEED = 42
if LOCAL_OR_COLAB == "LOCAL":
    DATA_DIR = "/home/juliana/internship_LINUX/datasets/EuroSAT_RGB"
else:

  # Set paths
  data_root = "/content/EuroSAT_RGB"
  zip_path = "/content/EuroSAT.zip"

  # Download and extract EuroSAT RGB dataset
  if not os.path.exists(data_root):
      print("Downloading EuroSAT RGB...")
      url = "https://madm.dfki.de/files/sentinel/EuroSAT.zip"

      # Create an unverified SSL context
      ssl._create_default_https_context = ssl._create_unverified_context

      urllib.request.urlretrieve(url, zip_path)

      print("Unzipping...")
      with zipfile.ZipFile(zip_path, 'r') as zip_ref:
          zip_ref.extractall("/content")
      os.rename("/content/2750", data_root)

  DATA_DIR = data_root
  print("EuroSAT RGB dataset downloaded and extracted.")


BATCH_SIZE = 32
NUM_EPOCHS = 2
LR = 1e-3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def get_data_loaders(data_dir, batch_size):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    dataset = datasets.ImageFolder(root=data_dir, transform=transform)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_set, test_set = random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=batch_size)
    return train_loader, test_loader, len(dataset.classes)

def build_model(num_classes, pretrained=False):
    model = resnet18(weights=None if not pretrained else "DEFAULT")
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model.to(DEVICE)

def train(model, loader, optimizer, criterion, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss, correct, total = 0, 0, 0
        for images, labels in loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {running_loss:.4f} - Train Accuracy: {acc:.2f}%")
    return model

def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    acc = 100 * correct / total
    print(f"Test Accuracy: {acc:.2f}%")
    return acc

def run_experiment(seed=SEED, freeze=False, pretrained=False):
    set_seed(seed)
    train_loader, test_loader, num_classes = get_data_loaders(DATA_DIR, BATCH_SIZE)

    model = build_model(num_classes, pretrained=pretrained)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LR)

    print("Training full model...")
    model = train(model, train_loader, optimizer, criterion, NUM_EPOCHS)
    torch.save(model.state_dict(), "resnet18_eurosat_from_scratch.pt")

    acc = evaluate(model, test_loader)

    if freeze:
        print("\nRunning linear probe...")
        model = build_model(num_classes)
        model.load_state_dict(torch.load("resnet18_eurosat_from_scratch.pt"))

        for param in model.parameters():
            param.requires_grad = False

        model.fc = nn.Linear(model.fc.in_features, num_classes)
        model = model.to(DEVICE)
        optimizer = optim.Adam(model.fc.parameters(), lr=LR)

        model = train(model, train_loader, optimizer, criterion, NUM_EPOCHS)
        acc = evaluate(model, test_loader)

    return acc

def run_multiple_experiments(n_runs=3, freeze=False, pretrained=False):
    accs = []
    for run in range(n_runs):
        print(f"\n========== Run {run + 1} ==========")
        acc = run_experiment(seed=SEED + run, freeze=freeze, pretrained=pretrained)
        accs.append(acc)
    print(f"\nMean Accuracy over {n_runs} runs: {np.mean(accs):.2f}%")
    print(f"Std Deviation: {np.std(accs):.2f}%")

# Run the code
if __name__ == "__main__":
    run_multiple_experiments(n_runs=3, freeze=True, pretrained=True)



EuroSAT RGB dataset downloaded and extracted.

========== Run 1 ==========


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 112MB/s]


Training full model...
Epoch [1/2] - Loss: 327.5162 - Train Accuracy: 85.35%
Epoch [2/2] - Loss: 189.3918 - Train Accuracy: 91.12%
Test Accuracy: 87.35%

Running linear probe...
Epoch [1/2] - Loss: 146.8580 - Train Accuracy: 93.25%
Epoch [2/2] - Loss: 111.1069 - Train Accuracy: 94.56%
Test Accuracy: 94.59%

========== Run 2 ==========
Training full model...
Epoch [1/2] - Loss: 323.6989 - Train Accuracy: 85.23%
Epoch [2/2] - Loss: 186.9981 - Train Accuracy: 91.20%
Test Accuracy: 90.93%

Running linear probe...
Epoch [1/2] - Loss: 127.1115 - Train Accuracy: 94.52%
Epoch [2/2] - Loss: 92.4459 - Train Accuracy: 95.59%
Test Accuracy: 95.56%

========== Run 3 ==========
Training full model...
Epoch [1/2] - Loss: 334.8889 - Train Accuracy: 84.96%
Epoch [2/2] - Loss: 196.8723 - Train Accuracy: 90.91%
Test Accuracy: 93.09%

Running linear probe...
Epoch [1/2] - Loss: 156.8873 - Train Accuracy: 93.07%
Epoch [2/2] - Loss: 116.4773 - Train Accuracy: 94.44%
Test Accuracy: 94.56%

Mean Accuracy over

In [3]:
# Run the code
if __name__ == "__main__":
    run_multiple_experiments(n_runs=3, freeze=True, pretrained=False)



========== Run 1 ==========
Training full model...
Epoch [1/2] - Loss: 696.4861 - Train Accuracy: 63.55%
Epoch [2/2] - Loss: 478.1807 - Train Accuracy: 74.59%
Test Accuracy: 67.61%

Running linear probe...
Epoch [1/2] - Loss: 409.4370 - Train Accuracy: 79.46%
Epoch [2/2] - Loss: 358.2151 - Train Accuracy: 81.39%
Test Accuracy: 84.19%

========== Run 2 ==========
Training full model...
Epoch [1/2] - Loss: 698.3165 - Train Accuracy: 63.62%
Epoch [2/2] - Loss: 483.3826 - Train Accuracy: 74.82%
Test Accuracy: 73.85%

Running linear probe...
Epoch [1/2] - Loss: 422.5408 - Train Accuracy: 78.42%
Epoch [2/2] - Loss: 361.0377 - Train Accuracy: 81.26%
Test Accuracy: 84.72%

========== Run 3 ==========
Training full model...
Epoch [1/2] - Loss: 703.3265 - Train Accuracy: 63.32%
Epoch [2/2] - Loss: 481.8541 - Train Accuracy: 74.77%
Test Accuracy: 76.22%

Running linear probe...
Epoch [1/2] - Loss: 390.5736 - Train Accuracy: 80.31%
Epoch [2/2] - Loss: 338.0528 - Train Accuracy: 82.50%
Test Accura